Simple least square solve with u.n_s = v_s.n_s

In [2]:
import numpy as np
import scipy.optimize

In [36]:
uInit00 = np.array([1, 1])
uInit01 = np.array([0, 1])
uInit02 = np.array([1, 0])
Uinit0 = np.concatenate([uInit00, uInit01, uInit02])
uInit10 = np.array([2, 2])
uInit11 = np.array([0, 2])
uInit12 = np.array([2, 0])
Uinit1 = np.concatenate([uInit10, uInit11, uInit12])
Uinit = np.concatenate([Uinit0, Uinit1])  # chosen intelligently

U = np.array([0]*Uinit.size) # don't know
print("U: ", U)

n00 = np.array([1, 0])
n01 = np.array([0, 1])
n02 = np.array([1, 1])
N0 = np.concatenate([n00, n01, n02])
n10 = np.array([-1, 0])
n11 = np.array([0, -1])
n12 = np.array([-1, -1])
N1 = np.concatenate([n10, n11, n12])
N = np.concatenate([N0, N1])  # given as such from C++ program

print("N: ", N)

V = N  # given as such from C++ program

nbPoints = N0.size // 2  # given as such from C++ program
nbFrames = N.size // (2 * nbPoints)  # given as such from C++ program

#######################################################


for frameId in range(nbFrames):
    UinitFrame = Uinit[(2*nbPoints)*frameId:(2*nbPoints)*(frameId+1)]
    NFrame = N[(2*nbPoints)*frameId:(2*nbPoints)*(frameId+1)]
    VFrame = V[(2*nbPoints)*frameId:(2*nbPoints)*(frameId+1)]
    A = np.zeros([nbPoints, 2*nbPoints])
    B = np.zeros([nbPoints])
    for pointId in range(nbPoints):
        A[pointId, 2*pointId:2*(pointId+1)] = NFrame[2*pointId:2*(pointId+1)]
        B[pointId] = np.dot(VFrame[2*pointId:2*(pointId+1)],
                            NFrame[2*pointId:2*(pointId+1)])

    print("cross product: ", A @ UinitFrame)
    print("B: ", B)

    cons = ({'type': 'eq', 'fun': lambda x:  A @ x - B})

    print(UinitFrame)
    if frameId == 0:
        Uframe = V[0:2*nbPoints]
    else:
        def f(x):
            timeGradient = x-U[(2*nbPoints)*(frameId-1):(2*nbPoints)*frameId]
            firstSpaceDimGradient = x[:2*(nbPoints-1)]-x[2:]
            secondSpaceDimGradient = 0 # to modify
            spaceGradient = firstSpaceDimGradient + secondSpaceDimGradient
            return np.sum(timeGradient**2) + np.sum(spaceGradient**2)
        resFrame = scipy.optimize.minimize(f,
                                           UinitFrame,
                                           constraints=cons)
        Uframe = resFrame.x
    print(Uframe)
    U[(2*nbPoints)*frameId:(2*nbPoints)*(frameId+1)] = Uframe


U:  [0 0 0 0 0 0 0 0 0 0 0 0]
N:  [ 1  0  0  1  1  1 -1  0  0 -1 -1 -1]
cross product:  [1. 1. 1.]
B:  [1. 1. 2.]
[1 1 0 1 1 0]
[1 0 0 1 1 1]
cross product:  [-2. -2. -2.]
B:  [1. 1. 2.]
[2 2 0 2 2 0]
[-1.         -0.49999995 -0.63636365 -1.         -0.90909091 -1.09090909]
